In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# HTML File lesen
folder = 'htmldata'
filename = 'Kommunalwahl Darmstadt 2026 Darmstadt _ VOTO.html'
with open(f"{folder}/{filename}", "r") as f:
    seite = f.read()

S = BeautifulSoup(seite, 'lxml')


In [3]:
# Hole alle Fragen (=cards)
all_divs = S.find_all('div')
cards = []
for d in all_divs:
    if len(d.get_attribute_list('data-slot')) > 0:
        if  d.get_attribute_list('data-slot')[0] == 'card':
            cards.append(d)

In [4]:
# Funktionen, um einzelne Elemente zu extrahieren
def get_shorttitle_from_card(card):
    for d in card.find_all("div"):
        if ('flex' in d.get_attribute_list("class")) and ('leading-relaxed' in d.get_attribute_list("class")):
            return d.find_all("div")[0].get_text()
        
def get_longtitle_from_card(card):
    for d in card.find_all("p"):
        if ('text-xl' in d.get_attribute_list("class")) and ('md:text-2xl' in d.get_attribute_list("class")):
            return d.get_text()
        
def get_parteien_from_card(card):
    parteien = []
    for d in card.find_all("div"):
        if ('justify-self-end' in d.get_attribute_list("class")) and ('md:text-lg' in d.get_attribute_list("class")):
            parteien.append(d.getText())

    return parteien

def get_vote_from_card(card):
    votes = []
    for d in card.find_all("div"):
        if ('align-middle' in d.get_attribute_list("class")) and ('md:leading-8' in d.get_attribute_list("class")):
            if len(d) > 0:
                votes.append(d.getText())
    return votes

In [5]:
# Iteriere über alle Cards und hole die Elemente
dfs = []
for c in cards:
    shorttitle = get_shorttitle_from_card(c)
    longtitle = get_longtitle_from_card(c)
    parteien = get_parteien_from_card(c)
    votes = get_vote_from_card(c)
    df_akt = pd.DataFrame({
        'parteien': parteien, 'votes': votes
    })
    df_akt['shorttitle'] = shorttitle
    df_akt['longtitle'] = longtitle
    dfs.append(df_akt)

In [6]:
df = pd.concat(dfs)
df = df.query("parteien != 'Ich'")
df.head()

,parteien,votes,shorttitle,longtitle
1,FDP,4,Abschaffung der Hundesteuer,Darmstadt sollte die Hundesteuer abschaffen.
2,Linke,3,Abschaffung der Hundesteuer,Darmstadt sollte die Hundesteuer abschaffen.
3,CDU,1,Abschaffung der Hundesteuer,Darmstadt sollte die Hundesteuer abschaffen.
4,Gemeinsame Liste,3,Abschaffung der Hundesteuer,Darmstadt sollte die Hundesteuer abschaffen.
5,DaGe,4,Abschaffung der Hundesteuer,Darmstadt sollte die Hundesteuer abschaffen.


In [7]:
df.to_excel("data/votes.xlsx", index=False)
df.to_csv("data/votes.csv", index=False)